# Подготовка базы данных

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect('window.db')

In [3]:
cursor = con.cursor()

In [4]:
sql = '''

create table employees (
    id integer primary key,
    name varchar(50),
    city varchar(50),
    department varchar(50),
    salary integer
);

insert into employees
(id, name, city, department, salary)
values
(24, 'Марина', 'Москва', 'it', 104),
(21, 'Елена', 'Самара', 'it', 84),
(22, 'Ксения', 'Москва', 'it', 90),
(25, 'Иван', 'Москва', 'it', 120),
(23, 'Леонид', 'Самара', 'it', 104),
(11, 'Дарья', 'Самара', 'hr', 70),
(12, 'Борис', 'Самара', 'hr', 78),
(31, 'Вероника', 'Москва', 'sales', 96),
(33, 'Анна', 'Москва', 'sales', 100),
(32, 'Григорий', 'Самара', 'sales', 96);

create table expenses (
    year integer,
    month integer,
    income integer,
    expense integer
);

insert into expenses
(year, month, income, expense)
values
(2020, 1, 94, 82),
(2020, 2, 94, 75),
(2020, 3, 94, 104),
(2020, 4, 100, 94),
(2020, 5, 100, 99),
(2020, 6, 100, 105),
(2020, 7, 100, 95),
(2020, 8, 100, 110),
(2020, 9, 104, 104);'''

In [5]:
cursor.executescript(sql)

OperationalError: table employees already exists

In [ ]:
sql = '''select * from employees'''

pd.read_sql(sql, con)

In [6]:
# Напишем функцию, чтобы удобнее было возвращать
def select(sql):
    return pd.read_sql(sql, con)

In [9]:
select(sql)

,id,name,department,salary,prev_salary,max_salary
0,11,Дарья,hr,70,70,78
1,12,Борис,hr,78,70,78
2,21,Елена,it,84,84,120
3,22,Ксения,it,90,84,120
4,23,Леонид,it,104,90,120
5,24,Марина,it,104,104,120
6,25,Иван,it,120,104,120
7,31,Вероника,sales,96,96,100
8,32,Григорий,sales,96,96,100
9,33,Анна,sales,100,96,100


# Задача 1

**Есть таблица сотрудников employees. Напишите запрос, который для каждого сотрудника выведет:**

* размер з/п предыдущего по зарплате сотрудника (среди коллег по департаменту);
* максимальную з/п по департаменту.


Если «предыдущего коллеги» нет (у сотрудника самая низкая зарплата в департаменте) — запрос должен возвращать зарплату самого сотрудника.

In [10]:
sql = '''
SELECT t.id, t.name, t.department, t.salary,
coalesce(lag(t.salary) over w, t.salary) as prev_salary,
max(t.salary) over w as max_salary
FROM employees as t
window w as (
partition by t.department

)
'''
select(sql)

,id,name,department,salary,prev_salary,max_salary
0,11,Дарья,hr,70,70,78
1,12,Борис,hr,78,70,78
2,21,Елена,it,84,84,120
3,22,Ксения,it,90,84,120
4,23,Леонид,it,104,90,120
5,24,Марина,it,104,104,120
6,25,Иван,it,120,104,120
7,31,Вероника,sales,96,96,100
8,32,Григорий,sales,96,96,100
9,33,Анна,sales,100,96,100


# Задача 2

**Есть таблица сотрудников employees. Для каждого человека мы хотим посчитать количество сотрудников, которые получают такую же или большую зарплату (ge_cnt):**

In [11]:
sql = '''
select t.id, t.name, t.salary,
round(cume_dist() over w  *10, 0) as ge_cnt
from employees as t
window w as (

order by t.salary desc
groups between current row and unbounded following
)

order by t.salary, t.id

'''
select(sql)

,id,name,salary,ge_cnt
0,11,Дарья,70,10.0
1,12,Борис,78,9.0
2,21,Елена,84,8.0
3,22,Ксения,90,7.0
4,31,Вероника,96,6.0
5,32,Григорий,96,6.0
6,33,Анна,100,4.0
7,23,Леонид,104,3.0
8,24,Марина,104,3.0
9,25,Иван,120,1.0


# Задача 3

**Есть таблица сотрудников employees. Для каждого человека мы хотим увидеть ближайшую большую зарплату (next_salary):**

In [12]:
sql = '''
select t.id, t.name, t.salary,
max(t.salary) over w as next_salary 
from employees as t
window w as (
order by t.salary
groups between 1 following and 1 following
)
order by t.salary, t.id

'''
select(sql)

,id,name,salary,next_salary
0,11,Дарья,70,78.0
1,12,Борис,78,84.0
2,21,Елена,84,90.0
3,22,Ксения,90,96.0
4,31,Вероника,96,100.0
5,32,Григорий,96,100.0
6,33,Анна,100,104.0
7,23,Леонид,104,120.0
8,24,Марина,104,120.0
9,25,Иван,120,NaN
